In [1]:
import sys
sys.path.append('../')

import pyterrier as pt
import pandas as pd
import os
from src.create_index import load_data, load_folds

In [2]:
sub_collections = [
    "t1",
    "t2",
    "t3",
    "t4",
    "t5",
]

In [3]:
def load_runs_for_subcollection(runs_path, sub_collection, fold_no):
    run_files = os.listdir(runs_path)
    runs = []
    run_names = []
    for run_file in run_files:
        if "_"+sub_collection in run_file and "F"+fold_no in run_file:
            run_name = run_file
            if "extended" in run_name or "long" in run_name:
                continue
        
            run_names.append(run_name)    
            run = pt.io.read_results(os.path.join(runs_path, run_name))
            runs.append(run)
    
    return runs, run_names

In [4]:
def make_table(results, folds=False):
    table = results.copy()
    
    
    # Rename runs
    table['name'] = table['name'].str.split('_').str[0]
    table = table.replace({"BM25": "BM25", "BM25+RM3": "+RM3", "BM25+Bo1": "+Bo1", "BM25+RF":"+RF", "BM25+qrel": "+qrel"})

    table = table.rename(columns={"name": "Run", "sub_collection":"t", "ndcg":"nDCG", "ndcg_cut.10":"nDCG@10", "P.10": "P@10", "recip_rank": "MRR"})
    table = table.replace({"t1": "$t_1$", "t2": "$t_2$", "t3": "$t_3$", "t4": "$t_4$", "t5": "$t_5$"})
    
    custom_order = ['BM25', '+RM3', '+Bo1', '+RF', '+qrel']
    table['Run'] = pd.Categorical(table['Run'], categories=custom_order, ordered=True)
    
    # Average over folds    
    if folds:
        table = table.drop("fold_no", axis=1).groupby(["t", "Run"]).mean()
    else:
        table = table.groupby(["t", "Run"]).mean()
           
    return table

# Natural Evolving

In [5]:
results = pd.DataFrame()

for sub_collection in sub_collections:
    topics, qrels = load_data(sub_collection)
    runs, run_names = load_runs_for_subcollection("../data/results_full", sub_collection, "0")
    
    res = pt.Experiment(
        runs,
        topics,
        qrels,
        names=run_names,
        eval_metrics=["ndcg", "ndcg_cut.10", "bpref", "P.10", "recip_rank"],
        verbose=True
        ) 
        
    res["sub_collection"] = sub_collection
    results = pd.concat([results, res])

Java started (triggered by _read_topics_trec) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
pt.Experiment: 100%|██████████| 5/5 [00:15<00:00,  3.04s/system]


In [7]:
table = make_table(results)
table

/tmp/ipykernel_153042/1755490063.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  table = table.groupby(["t", "Run"]).mean()


nDCG   nDCG@10     bpref      P@10       MRR
t     Run                                                    
$t_1$ BM25   0.287587  0.180019  0.315905  0.094785  0.259452
      +RM3   0.284912  0.170582  0.315886  0.091270  0.240171
      +Bo1   0.289766  0.177348  0.321397  0.094785  0.246114
      +RF    0.333804  0.228114  0.367402  0.109070  0.343640
      +qrel  0.322639  0.223734  0.333968  0.109977  0.322048
$t_2$ BM25   0.298741  0.182955  0.317212  0.103684  0.264393
      +RM3   0.298110  0.183684  0.314662  0.106501  0.256755
      +Bo1   0.307314  0.190286  0.323448  0.108126  0.270709
      +RF    0.358324  0.243108  0.371986  0.126219  0.375145
      +qrel  0.339727  0.233624  0.339201  0.124865  0.347611
$t_3$ BM25   0.363417  0.196333  0.435537  0.161269  0.367518
      +RM3   0.368467  0.189715  0.451861  0.157262  0.346519
      +Bo1   0.371169  0.195580  0.447305  0.162938  0.356804
      +RF    0.401135  0.230595  0.477078  0.183973  0.452345
      +qrel  0.382905  0.221027  0.447103  0.177129  0.416662
$t_4$ BM25   0.280586  0.168227  0.355423  0.136634  0.313497
      +RM3   0.286439  0.173707  0.372812  0.144307  0.306519
      +Bo1   0.293482  0.176806  0.373271  0.143317  0.326791
      +RF    0.331488  0.220423  0.399590  0.166832  0.451530
      +qrel  0.340078  0.245616  0.376504  0.180198  0.459123
$t_5$ BM25   0.210714  0.141878  0.279071  0.107642  0.311991
      +RM3   0.216000  0.144884  0.284948  0.111858  0.305309
      +Bo1   0.214374  0.142820  0.284366  0.110211  0.303804
      +RF    0.238310  0.170061  0.300885  0.126153  0.385099
      +qrel  0.253426  0.198917  0.292638  0.145059  0.436735

In [8]:
table.to_latex("../paper/table-results.tex", index=False, float_format="%.3f")

# Cross Validation

In [9]:
results = pd.DataFrame()

folds = load_folds()

for sub_collection in sub_collections:    
    topics, qrels = load_data(sub_collection)
    
    for fold_no in range(0, len(folds[sub_collection])):
        train_docids = folds[sub_collection][str(fold_no)]["train"]
        test_docids = folds[sub_collection][str(fold_no)]["test"]
        
        runs, run_names = load_runs_for_subcollection("../data/results", sub_collection, str(fold_no))
    
        qrels = qrels[~qrels["docno"].isin(train_docids)]
        
        res = pt.Experiment(
        runs,
        topics,
        qrels,
        names=run_names,
        eval_metrics=["ndcg", "ndcg_cut.10", "bpref", "P.10", "recip_rank"],
        verbose=True
        ) 
        
        res["sub_collection"] = sub_collection
        res["fold_no"] = fold_no
        results = pd.concat([results, res])
    

pt.Experiment: 100%|██████████| 5/5 [00:14<00:00,  2.94s/system]


In [10]:
make_table(results, folds=True)

/tmp/ipykernel_198308/1755490063.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  table = table.drop("fold_no", axis=1).groupby(["t", "Run"]).mean()


nDCG   nDCG@10     bpref      P@10       MRR
t     Run                                                    
$t_1$ BM25   0.201073  0.130325  0.221094  0.063303  0.169373
      +RM3   0.201639  0.124286  0.219087  0.060544  0.160161
      +Bo1   0.204611  0.127288  0.224692  0.062094  0.162361
      +RF    0.200338  0.129347  0.220243  0.063076  0.168661
      +qrel  0.201073  0.130325  0.221094  0.063303  0.169373
$t_2$ BM25   0.237800  0.147111  0.248171  0.073701  0.200901
      +RM3   0.240369  0.150867  0.252741  0.078149  0.194281
      +Bo1   0.246815  0.154427  0.256868  0.077028  0.207875
      +RF    0.235419  0.143647  0.249341  0.072761  0.196226
      +qrel  0.217929  0.135181  0.225487  0.069327  0.186593
$t_3$ BM25   0.261280  0.146112  0.323761  0.109176  0.265130
      +RM3   0.265378  0.141820  0.337308  0.107042  0.249720
      +Bo1   0.265383  0.142054  0.332093  0.109402  0.251008
      +RF    0.260606  0.144046  0.324159  0.108666  0.261536
      +qrel  0.267064  0.148077  0.331373  0.109508  0.266165
$t_4$ BM25   0.165245  0.104236  0.211717  0.078166  0.190928
      +RM3   0.172053  0.109466  0.223241  0.083520  0.191126
      +Bo1   0.176907  0.111075  0.225333  0.081594  0.206177
      +RF    0.164268  0.102837  0.210711  0.077413  0.189664
      +qrel  0.167958  0.106136  0.216451  0.078788  0.192573
$t_5$ BM25   0.164469  0.113394  0.219474  0.081190  0.240259
      +RM3   0.170865  0.117983  0.227460  0.085634  0.239525
      +Bo1   0.168652  0.115046  0.226699  0.083201  0.235748
      +RF    0.164967  0.113414  0.220867  0.081103  0.241293
      +qrel  0.166095  0.114318  0.222110  0.081383  0.240844

In [15]:
table.to_latex("../paper/table-results-fold.tex", index=False, float_format="%.3f")
table